In [ ]:
#@title Model

model = '2.2-diffusers' #@param ["2.0", "2.1", "2.1-diffusers", "2.2-diffusers", "3.0", "3.0-diffusers", "3.1"]

In [ ]:
#@title Settings

branch = 'main' #@param ["main", "dev"]
checkout_to_commit = "" #@param {type:"string"}
use_ngrok = False #@param {type:"boolean"}
ngrok_auth_key = "" #@param {type:"string"}
use_localtunnel = False #@param {type:"boolean"}
use_cloudflared = False #@param {type:"boolean"}
use_flash_attention = True #@param {type:"boolean"}
save_models_to_gdrive = False #@param {type:"boolean"}
save_images_to_gdrive = False #@param {type:"boolean"}

use_diffusers = False

if model == "2.0":
  model_name = "kd20"

if model == "2.1":
  model_name = "kd21"

if model == "2.1-diffusers":
  model_name = "kd21"
  use_diffusers = True

if model == "2.2-diffusers":
  model_name = "kd22"
  use_diffusers = True

if model == "3.0":
  model_name = "kd30"

if model == "3.0-diffusers":
  model_name = "kd30"
  use_diffusers = True
  
if model == "3.1":
  model_name = "kd31"

In [ ]:
#@title Diffusers
optimizations = []

half_weights = True #@param {type:"boolean"}
use_xformers = False #@param {type:"boolean"}
use_sliced_attention = False #@param {type:"boolean"}
use_sequential_offload = False #@param {type:"boolean"}
channels_last_memory = False #@param {type:"boolean"}

if half_weights: optimizations.append('half_weights')
if use_xformers: optimizations.append('xformers')
if use_sliced_attention: optimizations.append('sliced_attention')
if use_sequential_offload: optimizations.append('sequential_offload')
if channels_last_memory: optimizations.append('channels_last')

optimizations = str.join(',', optimizations)

In [ ]:
#@title Extensions
disabled_extensions = []

image_browser = True #@param {type:"boolean"}
image_tools = False #@param {type:"boolean"}
video_tools = False #@param {type:"boolean"}
interrogator = False #@param {type:"boolean"}
mesh_gen = False #@param {type:"boolean"}
multi_view = False #@param {type:"boolean"}
animation = False #@param {type:"boolean"}
video = False #@param {type:"boolean"}
image_editor = False #@param {type:"boolean"}
image_to_video = False #@param {type:"boolean"}
prompt_styles = True #@param {type:"boolean"}
segmentation = False #@param {type:"boolean"}
upscaler = True #@param {type:"boolean"}
networks = True #@param {type:"boolean"}
training = False #@param {type:"boolean"}
bg_remover = False #@param {type:"boolean"}
llm_enhancer = False #@param {type:"boolean"}
pipeline_enhancer = False #@param {type:"boolean"}
stable_cascade = False #@param {type:"boolean"}
pixart = False #@param {type:"boolean"}
kwai_colors = False #@param {type:"boolean"}
flux = False #@param {type:"boolean"}
sana = False #@param {type:"boolean"}
switti = False #@param {type:"boolean"}

if not image_browser: disabled_extensions.append('kd-image-browser')
if not image_tools: disabled_extensions.append('kd-image-tools')
if not video_tools: disabled_extensions.append('kd-video-tools')
if not interrogator: disabled_extensions.append('kd-interrogator')
if not mesh_gen: disabled_extensions.append('kd-mesh-gen')
if not multi_view: disabled_extensions.append('kd-multi-view')
if not animation: disabled_extensions.append('kd-animation')
if not video: disabled_extensions.append('kd-video')
if not image_editor: disabled_extensions.append('kd-image-editor')
if not image_to_video: disabled_extensions.append('kd-image-to-video')
if not prompt_styles: disabled_extensions.append('kd-prompt-styles')
if not segmentation: disabled_extensions.append('kd-segmentation')
if not upscaler: disabled_extensions.append('kd-upscaler')
if not networks: disabled_extensions.append('kd-networks')
if not training: disabled_extensions.append('kd-training')
if not bg_remover: disabled_extensions.append('kd-bg-remover')
if not llm_enhancer: disabled_extensions.append('kd-llm-enhancer')
if not pipeline_enhancer: disabled_extensions.append('kd-pipeline-enhancer')
if not stable_cascade: disabled_extensions.append('kd-stable-cascade')
if not pixart: disabled_extensions.append('kd-pixart')
if not kwai_colors: disabled_extensions.append('kd-kwai-kolors')
if not flux: disabled_extensions.append('kd-flux')
if not sana: disabled_extensions.append('kd-sana')
if not switti: disabled_extensions.append('kd-switti')

disabled_extensions = str.join(',', disabled_extensions)

In [ ]:
#@title UI

side_tabs_view = True #@param {type:"boolean"}

In [ ]:
if save_models_to_gdrive or save_images_to_gdrive:
  from google.colab import drive

  drive.mount('/content/gdrive')

In [ ]:
!python --version

[gpu] = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print (gpu)

import torch
print(torch.__version__)

!nvidia-smi

In [ ]:
%cd /content

!git clone -b {branch} https://github.com/seruva19/kubin.git
!git clone https://github.com/seruva19/kubin-extensions.git /content/kubin/extensions
%cd /content/kubin

if checkout_to_commit != '':
  !git checkout {checkout_to_commit}

!pip install --upgrade -r requirements.txt

if use_flash_attention:
  !pip install /content/kubin/wheels/flash_attn-1.0.9-cp311-cp311-linux_x86_64.whl

if use_xformers:
  !pip install xformers==0.0.22.post7
  !python -m xformers.info

In [ ]:
if use_ngrok:
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_auth_key)

  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

  print(ngrok.connect(7860, "http"))

In [ ]:
if use_localtunnel:
    !npm install -g localtunnel
    lturl = !curl ipv4.icanhazip.com
    print("endpoint IP for localtunnel:")
    print(lturl[0])

In [ ]:
if use_cloudflared:
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

In [ ]:
!kill -9 $(lsof -t -i tcp:7860)
%cd /content/kubin

!python src/kubin.py \
  --model-name={model_name} \
  --cache-dir={"/content/gdrive/MyDrive/kubin/models" if save_models_to_gdrive else "/content/kubin/models"} \
  --output-dir={"/content/gdrive/MyDrive/kubin/output" if save_images_to_gdrive else "/content/kubin/output"} \
  --disabled-extensions={disabled_extensions} \
  --pipeline={"diffusers" if use_diffusers else "native"} \
  --optimize={optimizations} \
  {"--side-tabs='use'" if side_tabs_view else ""} \
  --share={"none" if use_ngrok or use_localtunnel or use_cloudflared else "gradio"} \
  {"--flash-attention='use'" if use_flash_attention else ""} & \
  {"npx localtunnel --port 7860" if use_localtunnel else ":"} & \
  {"cloudflared tunnel --url http://localhost:7860" if use_cloudflared else ":"}